In [ ]:
import json
import tqdm
from lxml import etree
from pathlib import Path
from pprint import pprint
import pandas as pd

from ddp_util import get_path_list


#fsdb_dir =  "../../data/db/mom-data/metadata.charter.public/AT-AWMK"
fsdb_dir =  "../../data/db/mom-data/metadata.charter.public"
cei_filename =  "cei.xml"


namespaces = {"atom": "http://www.w3.org/2005/Atom", "cei": "http://www.monasterium.net/NS/cei"}

"""
TODO: make module for this: xpath_expressions could just be a json that is filled before
mapping_file_dir = "../../data/mapping/xpath_selection.json"
with open(mapping_file_dir, mode="r") as mapping_file:
    xpath_expressions = json.load(mapping_file)
"""

xpath_expressions = {
    "atom_entry": "/atom:entry/atom:id/text()",
    "weird_c": "//cei:c/text()",
    "cei_sigillant": "//cei:sigillant/text()",
    "cei_seal_ATTRIBUTE_place": "//cei:seal/@place",
    "cei_legend_ATTRIBUTE_place": "//cei:legend/@place",
    "cei_listBibl": "//cei:listBibl/text()",
    "cei_bibl": "//cei:bibl/text()",
    "cei_pTenor": "//cei:pTenor/text()",
    "cei_tenor": "//cei:tenor/text()",
    "cei_tenor_desc": "//cei:tenor/descendant-or-self::*/text()",
    "cei_tenor_node": "//cei:tenor/node()",
    "cei_mod_type": "//cei:mod/@type",
    "cei_supplied_lang": "//cei:supplied/@lang"
}


def parse_cei(cei_path):
    with open(cei_path, "rb") as f:
        root = etree.parse(f).getroot()
        data = {}
        for key, xpath_expr in xpath_expressions.items():
            result = root.xpath(xpath_expr, namespaces=namespaces, smart_strings=False)
            if len(result) == 1:
                data[key] = result[0] or None
            else:
                data[key] = result or None
        return data

In [ ]:
charter_paths = get_path_list(fsdb_dir, cei_filename)

newList = []

for charter_path in tqdm.tqdm(charter_paths):
    data = parse_cei(charter_path)
    newList.append(data)

In [ ]:
df = pd.DataFrame.from_records(newList)

In [ ]:
df